In [ ]:
from series import Series

single_stories_1 = Series('Single Stories 1', 'DP000732')
#single_stories_1.download_xml_subtitles()
single_stories_1.convert_xml_to_srt()
#single_stories_1.normalize_srt()


In [6]:
from series import Series

single_stories_3 = Series('Single Stories 3', 'DP000743')
single_stories_3.download_xml_subtitles()
#single_stories_3.convert_xml_to_srt()
#single_stories_1.normalize_srt()


File downloaded successfully: output/xml/single-stories-1/1.xml
File downloaded successfully: output/xml/single-stories-1/2.xml
File downloaded successfully: output/xml/single-stories-1/3.xml
File downloaded successfully: output/xml/single-stories-1/4.xml
File downloaded successfully: output/xml/single-stories-1/5.xml
File downloaded successfully: output/xml/single-stories-1/6.xml
File downloaded successfully: output/xml/single-stories-1/7.xml
File downloaded successfully: output/xml/single-stories-1/8.xml
File downloaded successfully: output/xml/single-stories-1/9.xml
File downloaded successfully: output/xml/single-stories-1/10.xml
File downloaded successfully: output/xml/single-stories-1/11.xml
File downloaded successfully: output/xml/single-stories-1/12.xml
File downloaded successfully: output/xml/single-stories-1/13.xml
File downloaded successfully: output/xml/single-stories-1/14.xml
File downloaded successfully: output/xml/single-stories-1/15.xml
File downloaded successfully: outp

# Series Extraction
Extract the series title and id from the homepage.
Since the first 6 series do not have subtitles, they will be ignored. 

In [ ]:
import requests
from bs4 import BeautifulSoup

homepage_url = 'https://chinese.littlefox.com/en/story'
try:
    response = requests.get(homepage_url)
    response.raise_for_status()
except requests.RequestException as e:
    print(f"Error fetching main URL: {e}")

soup = BeautifulSoup(response.text, 'html.parser')
contents_divs = soup.find_all('div', class_='constents_wrap')
series = []

for div in contents_divs[6:]:
    id = div['data-smid']
    title = div.find('div', class_='thumb_titl').find('a').text
    print(title)
    series.append(Series(title, id))

Single Stories
Mrs. Kelly's Class
Who Am I?
Where Am I?
The Big Green Forest
Bat and Friends
Car School
Dino Buddies
Single Stories
Space Patrol
Peter Rabbit and Friends
Wacky Ricky 
Magic Marker
Bird and Kip
Sam and Lucky 
Meet the Animals
Single Stories
Jack and the Beanstalk
Puss in Boots
Snow White and the Seven Dwarfs 
Cinderella
Hana's Album
Sunshine School
Wizard and Cat 
The Carter Family
Single Stories
Fun at Kids Central
 The Light Princess
Rocket Girl
Danny's Adventures
The Wishing Well
Thumbelina
Single Stories
Rocket Girl's Journey to the West
Journey to the West
Alice's Adventures in Wonderland
The Jungle Book
The Railway Children
The Willow Creek Twins
The Little Mermaid
Aladdin and His Wonderful Lamp
Pinocchio
